In [12]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

from torch import nn
from torch.nn import functional as F

import sys 
import getpass

sys.path.append('../')
from Miniproject_2.model import *
torch.set_grad_enabled(True);

In [13]:
def test_init_weights(model):
    if isinstance(model,nn.Conv2d) or isinstance(model,nn.ConvTranspose2d):
        model.weight.data.normal_(0,0.5, generator=torch.manual_seed(0))
        
def compare(x,y,decimals=7):
    return torch.all(torch.round(torch.abs(x - y), decimals=decimals)==0.).item()

## My Sequential 

**REMEMBER TO PUT A FIXED SEED IN MODEL.PY**

In [3]:
stride = ks = 2

In [4]:
my_conv1 = Conv2d(in_channels=3, out_channels=5, stride=stride,  kernel_size=ks)
my_conv2 = Conv2d(in_channels=5, out_channels=5, stride=stride,  kernel_size=ks)


my_tconv1 = TransposeConv2d(in_channels=5, out_channels=5,  kernel_size=ks, stride=stride, padding=0, dilation=1)
my_tconv2 = TransposeConv2d(in_channels=5, out_channels=3,  kernel_size=ks, stride=stride, padding=0, dilation=1)

my_relu = ReLU()
my_sigmoid = Sigmoid()

my_Net = Sequential(my_conv1, my_relu, my_conv2, my_relu, my_tconv1, my_relu, my_tconv2, my_sigmoid)


In [5]:
valid_input, valid_target = torch.load('../val_data.pkl')
select = 10
x = (valid_input[:select].float())
t = (valid_target[:select].float())

y = my_Net(x)
loss = MSE()
loss(y,t)
my_Net.backward(loss.backward())

In [6]:
my_parameters = [list(my_Net.parameters())[::2][i][0] for i in range(4)]
my_gradients  = [list(my_Net.parameters())[::2][i][1] for i in range(4)]

## nn Sequential

In [7]:
stride = ks = 2

conv1 = nn.Conv2d(in_channels=3, out_channels=5,  kernel_size=ks, stride=stride, bias=False)
conv2 = nn.Conv2d(in_channels=5, out_channels=5,  kernel_size=ks, stride=stride, bias=False)

tconv1 = nn.ConvTranspose2d(in_channels=5, out_channels=5,  kernel_size=ks, stride=stride, padding=0, dilation=1, bias=False)
tconv2 = nn.ConvTranspose2d(in_channels=5, out_channels=3,  kernel_size=ks, stride=stride, padding=0, dilation=1, bias=False)

relu = nn.ReLU()
sigmoid = nn.Sigmoid()

Net = nn.Sequential(conv1, relu, conv2, relu, tconv1, relu, tconv2, sigmoid)
Net.apply(test_init_weights);

In [8]:
valid_input, valid_target = torch.load('../val_data.pkl')
criterion = nn.MSELoss()

select = 10
x = (valid_input[:select].float()).requires_grad_()
t = (valid_target[:select].float())

y = Net(x)
loss = criterion(y, t)
loss.backward()

gradients = []
with torch.no_grad():
    for p in Net.parameters(): gradients.append(p.grad)

## Comparison

In [9]:
parameters = [list(Net.parameters())[i] for i in range(4)]

In [10]:
[torch.all(parameters[0]==my_parameters[0]).item() for i in range(4)]

[True, True, True, True]

In [11]:
[compare(gradients[0], my_gradients[0], decimals=4) for i in range(4)]

[True, True, True, True]